## 연습문제 8번

In [1]:
import tensorflow as tf
import numpy as np

/Users/JOLEE/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# 데이터 불러오기
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# 이번에는 데이터를 0~4까지만 씀
mask = mnist.train.labels < 5
X_train = mnist.train.images[mask]
y_train = mnist.train.labels[mask]

mask_val = mnist.validation.labels < 5
X_val = mnist.validation.images[mask_val]
y_val = mnist.validation.labels[mask_val]

In [4]:
# 그래프 환경설정
n_train_samples = X_train.shape[0]
input_dim  = X_train.shape[-1]
n_hidden = 100
n_class = 5

learning_rate = 0.001

In [41]:
# 그래프 만들기

tf.reset_default_graph()

with tf.name_scope("input"):
    X = tf.placeholder(dtype=tf.float32, shape=(None, input_dim), name="X")
    y = tf.placeholder(dtype=tf.int64, shape=(None), name="y")

from functools import partial 
he_init = tf.contrib.layers.variance_scaling_initializer()
custom_hidden_layer = partial(tf.layers.dense, units=n_hidden,
                              activation=tf.nn.elu, kernel_initializer=he_init)

hidden1 = custom_hidden_layer(inputs=X, name="hidden1")
hidden2 = custom_hidden_layer(hidden1, name="hidden2")
hidden3 = custom_hidden_layer(hidden2, name="hidden3")
hidden4 = custom_hidden_layer(hidden3, name="hidden4")
hidden5 = custom_hidden_layer(hidden4, name="hidden5")
logits = tf.layers.dense(hidden5, n_class, name="logits_output")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, 
                                                              logits=logits, name="cross-entropy")
    loss = tf.reduce_mean(xentropy, name = "loss")

with tf.name_scope("eval"):
    y_pred = tf.argmax(logits, axis=1, name="prediction")
    correct = tf.equal(y_pred, y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
with tf.name_scope("training"):
    global_step = tf.train.get_or_create_global_step()
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss, global_step=global_step)


In [42]:
## 학습
# 환경 설정
n_epoch = 1000
batch_size = 10000



# Summary
from datetime import datetime
now = datetime.now().strftime("%Y%m%d%H%M%S")
logdir = "./ch11/log/run-{}".format(now)
loss_summary = tf.summary.scalar("loss", loss) # scalar 를 그래프에 먼저 추가해주고 file_writer 만들어야 하는듯..? 아닌가?
with tf.name_scope("accuracy"):
    val_acc_summary = tf.summary.scalar("validation", accuracy)
    train_acc_summary = tf.summary.scalar("train", accuracy)
file_writer = tf.summary.FileWriter(logdir = logdir, graph = tf.get_default_graph()) 


#Initializer
init = tf.global_variables_initializer()

# Saver
saver = tf.train.Saver() # 무조건 맨 마지막에 나와야하는 느낌..?

with tf.Session() as sess:
    sess.run(init)
    # 조기 종료 변수
    distance_to_best = 0
    best_val_acc = 0.0
    for epoch in range(n_epoch):
        new_idx = np.random.permutation(n_train_samples)
        X_train = X_train[new_idx]
        y_train = y_train[new_idx]
        
        for batch in range( n_train_samples // batch_size ):
            start = batch * batch_size
            end = (batch+1) * batch_size
            feed_dict = {X: X_train[start:end], y: y_train[start:end]}
            
            sess.run(training_op, feed_dict=feed_dict)
            
            # tensorboard summary 기록
            if batch % 10 == 0:
                loss_val, train_acc_summ_eval = sess.run([loss_summary, train_acc_summary], feed_dict=feed_dict)
                file_writer.add_summary(loss_val, tf.train.global_step(sess, global_step))
                file_writer.add_summary(train_acc_summ_eval, tf.train.global_step(sess, global_step))

        # Accuracy Evaluation
        if epoch % 10 == 0:
            loss_eval, train_acc_eval, val_acc_summ_eval = sess.run([loss, accuracy, val_acc_summary], feed_dict=feed_dict)
            val_acc_eval = accuracy.eval({X:X_val, y:y_val})
            file_writer.add_summary(val_acc_summ_eval, tf.train.global_step(sess, global_step))
            
            print(epoch, "Loss:", loss.eval(feed_dict), "Train Acc:", train_acc_eval, "Val Acc:", val_acc_eval)
            if val_acc_eval > best_val_acc :
                distance_to_best = 0
                best_val_acc = val_acc_eval
                save_path = saver.save(sess, "ch11/model/practice8_best_model.ckpt")
            else:
                distance_to_best += 1
        # 조기 종료
        if distance_to_best > 2:
            print("Eearly Stopping..")
            break

    save_path = saver.save(sess, "ch11/model/practice8_model_final.ckpt")
    file_writer.close() # 꼭 닫아줘야 텐서보드에 잘 나타나는듯

0 Loss: 0.857849 Train Acc: 0.7268 Val Acc: 0.733776
10 Loss: 0.12574 Train Acc: 0.9612 Val Acc: 0.964816
20 Loss: 0.088523 Train Acc: 0.9713 Val Acc: 0.973417
30 Loss: 0.0561811 Train Acc: 0.9827 Val Acc: 0.978108
40 Loss: 0.0440628 Train Acc: 0.9873 Val Acc: 0.982408
50 Loss: 0.0312323 Train Acc: 0.9905 Val Acc: 0.983972
60 Loss: 0.0294698 Train Acc: 0.9913 Val Acc: 0.986317
70 Loss: 0.0209395 Train Acc: 0.9945 Val Acc: 0.987881
80 Loss: 0.0138324 Train Acc: 0.997 Val Acc: 0.988663
90 Loss: 0.012703 Train Acc: 0.9973 Val Acc: 0.989054
100 Loss: 0.0111329 Train Acc: 0.9976 Val Acc: 0.988272
110 Loss: 0.00702966 Train Acc: 0.9987 Val Acc: 0.989054
120 Loss: 0.0056725 Train Acc: 0.9992 Val Acc: 0.989054
Eearly Stopping..


### Batch Normalization 삽입

In [43]:
# 그래프 만들기

tf.reset_default_graph()

with tf.name_scope("input"):
    X = tf.placeholder(dtype=tf.float32, shape=(None, input_dim), name="X")
    y = tf.placeholder(dtype=tf.int64, shape=(None), name="y")
    is_training = tf.placeholder(tf.bool, shape=(), name="is_training")

from functools import partial 
he_init = tf.contrib.layers.variance_scaling_initializer()

layer_params = {
    "units": n_hidden,
    "activation": tf.nn.elu,
    "kernel_initializer": he_init
}
bn_params = {
    'training': is_training,
    'momentum': 0.99
}
def dense_with_batch_layer(inputs, bn_params, units, activation, kernel_initializer, name):
    with tf.variable_scope(name):
        dense = tf.layers.dense(inputs, units=units, activation=None, 
                        kernel_initializer=kernel_initializer, name="dense")
        norm = tf.layers.batch_normalization(dense, name="batchnorm", **bn_params)
        outputs = activation(norm, name="activation")
        return outputs

custom_hidden_layer = partial(dense_with_batch_layer, bn_params=bn_params, **layer_params)

hidden1 = custom_hidden_layer(inputs=X, name="hidden1")
hidden2 = custom_hidden_layer(hidden1, name="hidden2")
hidden3 = custom_hidden_layer(hidden2, name="hidden3")
hidden4 = custom_hidden_layer(hidden3, name="hidden4")
hidden5 = custom_hidden_layer(hidden4, name="hidden5")
logits = tf.layers.dense(hidden5, n_class, name="logits_output")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, 
                                                              logits=logits, name="cross-entropy")
    loss = tf.reduce_mean(xentropy, name = "loss")

with tf.name_scope("eval"):
    y_pred = tf.argmax(logits, axis=1, name="prediction")
    correct = tf.equal(y_pred, y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
with tf.name_scope("training"):
    global_step = tf.train.get_or_create_global_step()
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops): # 이거 안해주면 moving average, variance update 안된다고 함
        training_op = optimizer.minimize(loss, global_step=global_step)


In [44]:
## 학습
# 환경 설정
n_epoch = 1000
batch_size = 10000

# Summary
from datetime import datetime
now = datetime.now().strftime("%Y%m%d%H%M%S")
logdir = "./ch11/log/run-{}".format(now)
loss_summary = tf.summary.scalar("loss", loss) # scalar 를 그래프에 먼저 추가해주고 file_writer 만들어야 하는듯..? 아닌가?
with tf.name_scope("accuracy"):
    val_acc_summary = tf.summary.scalar("validation", accuracy)
    train_acc_summary = tf.summary.scalar("train", accuracy)
file_writer = tf.summary.FileWriter(logdir = logdir, graph = tf.get_default_graph()) 


#Initializer
init = tf.global_variables_initializer()

# Saver
saver = tf.train.Saver() # 무조건 맨 마지막에 나와야하는 느낌..?

with tf.Session() as sess:
    sess.run(init)
    # 조기 종료 변수
    distance_to_best = 0
    best_val_acc = 0.0
    for epoch in range(n_epoch):
        new_idx = np.random.permutation(n_train_samples)
        X_train = X_train[new_idx]
        y_train = y_train[new_idx]
        
        for batch in range( n_train_samples // batch_size ):
            start = batch * batch_size
            end = (batch+1) * batch_size
            feed_dict = {X: X_train[start:end], y: y_train[start:end], is_training: True}
            sess.run(training_op, feed_dict=feed_dict)
            
            # tensorboard summary 기록
            if batch % 10 == 0:
                loss_val, train_acc_summ_eval = sess.run([loss_summary, train_acc_summary], feed_dict=feed_dict)
                file_writer.add_summary(loss_val, tf.train.global_step(sess, global_step))
                file_writer.add_summary(train_acc_summ_eval, tf.train.global_step(sess, global_step))
        
        # Accuracy Evaluation
        if epoch % 10 == 0:
            feed_dict[is_training] = False
            loss_eval, train_acc_eval, val_acc_summ_eval = sess.run([loss, accuracy, val_acc_summary], feed_dict=feed_dict)
            val_acc_eval = accuracy.eval({X:X_val, y:y_val, is_training:False})
            file_writer.add_summary(val_acc_summ_eval, tf.train.global_step(sess, global_step))
            
            print(epoch, "Loss:", loss_eval, "Train Acc:", train_acc_eval, "Val Acc:", val_acc_eval)
            if val_acc_eval > best_val_acc :
                distance_to_best = 0
                best_val_acc = val_acc_eval
                save_path = saver.save(sess, "ch11/model/practice8_best_model.ckpt")
            else:
                distance_to_best += 1
        # 조기 종료
        if distance_to_best > 2:
            print("Eearly Stopping..")
            break

    save_path = saver.save(sess, "ch11/model/practice8_model_final.ckpt")
    file_writer.close() # 꼭 닫아줘야 텐서보드에 잘 나타나는듯

0 Loss: 0.826876 Train Acc: 0.7016 Val Acc: 0.696247
10 Loss: 0.143745 Train Acc: 0.9582 Val Acc: 0.960125
20 Loss: 0.0868205 Train Acc: 0.9747 Val Acc: 0.974199
30 Loss: 0.0697867 Train Acc: 0.9806 Val Acc: 0.981626
40 Loss: 0.0575586 Train Acc: 0.9855 Val Acc: 0.982408
50 Loss: 0.0456887 Train Acc: 0.9889 Val Acc: 0.983972
60 Loss: 0.0370139 Train Acc: 0.9912 Val Acc: 0.984754
70 Loss: 0.0295422 Train Acc: 0.9936 Val Acc: 0.984754
80 Loss: 0.0224689 Train Acc: 0.9958 Val Acc: 0.985536
90 Loss: 0.0167428 Train Acc: 0.9975 Val Acc: 0.986317
100 Loss: 0.0118597 Train Acc: 0.9988 Val Acc: 0.987881
110 Loss: 0.00889411 Train Acc: 0.9991 Val Acc: 0.989054
120 Loss: 0.00648587 Train Acc: 0.9995 Val Acc: 0.989445
130 Loss: 0.00526068 Train Acc: 0.9999 Val Acc: 0.989445
140 Loss: 0.00402384 Train Acc: 1.0 Val Acc: 0.989836
150 Loss: 0.00312073 Train Acc: 1.0 Val Acc: 0.990227
160 Loss: 0.00270656 Train Acc: 0.9999 Val Acc: 0.990227
170 Loss: 0.00213576 Train Acc: 1.0 Val Acc: 0.990618
180 Los

### Dropout 삽입

In [45]:
# 그래프 만들기

tf.reset_default_graph()

with tf.name_scope("input"):
    X = tf.placeholder(dtype=tf.float32, shape=(None, input_dim), name="X")
    y = tf.placeholder(dtype=tf.int64, shape=(None), name="y")
    is_training = tf.placeholder(tf.bool, shape=(), name="is_training")

from functools import partial 
he_init = tf.contrib.layers.variance_scaling_initializer()

layer_params = {
    "units": n_hidden,
    "activation": tf.nn.elu,
    "kernel_initializer": he_init
}
bn_params = {
    'training': is_training,
    'momentum': 0.99
}
def dense_with_bn_and_dropout_layer(inputs, bn_params, units, activation, kernel_initializer, name):
    with tf.variable_scope(name):
        dense = tf.layers.dense(inputs, units=units, activation=None, 
                        kernel_initializer=kernel_initializer, name="dense")
        norm = tf.layers.batch_normalization(dense, name="batchnorm", **bn_params)
        outputs = activation(norm, name="activation")
        return tf.layers.dropout(outputs) # 이렇게 넣어주면 되겠지?

custom_hidden_layer = partial(dense_with_bn_and_dropout_layer, bn_params=bn_params, **layer_params)

hidden1 = custom_hidden_layer(inputs=X, name="hidden1")
hidden2 = custom_hidden_layer(hidden1, name="hidden2")
hidden3 = custom_hidden_layer(hidden2, name="hidden3")
hidden4 = custom_hidden_layer(hidden3, name="hidden4")
hidden5 = custom_hidden_layer(hidden4, name="hidden5")
logits = tf.layers.dense(hidden5, n_class, name="logits_output")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, 
                                                              logits=logits, name="cross-entropy")
    loss = tf.reduce_mean(xentropy, name = "loss")

with tf.name_scope("eval"):
    y_pred = tf.argmax(logits, axis=1, name="prediction")
    correct = tf.equal(y_pred, y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
with tf.name_scope("training"):
    global_step = tf.train.get_or_create_global_step()
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        training_op = optimizer.minimize(loss, global_step=global_step)


In [46]:
## 학습
# 환경 설정
n_epoch = 1000
batch_size = 10000

# Summary
from datetime import datetime
now = datetime.now().strftime("%Y%m%d%H%M%S")
logdir = "./ch11/log/run-{}".format(now)
loss_summary = tf.summary.scalar("loss", loss) # scalar 를 그래프에 먼저 추가해주고 file_writer 만들어야 하는듯..? 아닌가?
with tf.name_scope("accuracy"):
    val_acc_summary = tf.summary.scalar("validation", accuracy)
    train_acc_summary = tf.summary.scalar("train", accuracy)
file_writer = tf.summary.FileWriter(logdir = logdir, graph = tf.get_default_graph()) 


#Initializer
init = tf.global_variables_initializer()

# Saver
saver = tf.train.Saver() # 무조건 맨 마지막에 나와야하는 느낌..?

with tf.Session() as sess:
    sess.run(init)
    # 조기 종료 변수
    distance_to_best = 0
    best_val_acc = 0.0
    for epoch in range(n_epoch):
        new_idx = np.random.permutation(n_train_samples)
        X_train = X_train[new_idx]
        y_train = y_train[new_idx]
        
        for batch in range( n_train_samples // batch_size ):
            start = batch * batch_size
            end = (batch+1) * batch_size
            feed_dict = {X: X_train[start:end], y: y_train[start:end], is_training: True}
            sess.run(training_op, feed_dict=feed_dict)
            
            # tensorboard summary 기록
            if batch % 10 == 0:
                loss_val, train_acc_summ_eval = sess.run([loss_summary, train_acc_summary], feed_dict=feed_dict)
                file_writer.add_summary(loss_val, tf.train.global_step(sess, global_step))
                file_writer.add_summary(train_acc_summ_eval, tf.train.global_step(sess, global_step))
        
        # Accuracy Evaluation
        if epoch % 10 == 0:
            feed_dict[is_training] = False
            loss_eval, train_acc_eval, val_acc_summ_eval = sess.run([loss, accuracy, val_acc_summary], feed_dict=feed_dict)
            val_acc_eval = accuracy.eval({X:X_val, y:y_val, is_training:False})
            file_writer.add_summary(val_acc_summ_eval, tf.train.global_step(sess, global_step))
            
            print(epoch, "Loss:", loss_eval, "Train Acc:", train_acc_eval, "Val Acc:", val_acc_eval)
            if val_acc_eval > best_val_acc :
                distance_to_best = 0
                best_val_acc = val_acc_eval
                save_path = saver.save(sess, "ch11/model/practice8_best_model.ckpt")
            else:
                distance_to_best += 1
        # 조기 종료
        if distance_to_best > 2:
            print("Eearly Stopping..")
            break

    save_path = saver.save(sess, "ch11/model/practice8_model_final.ckpt")
    file_writer.close() # 꼭 닫아줘야 텐서보드에 잘 나타나는듯

0 Loss: 0.784087 Train Acc: 0.7373 Val Acc: 0.73534
10 Loss: 0.204651 Train Acc: 0.9375 Val Acc: 0.942142
20 Loss: 0.0989285 Train Acc: 0.971 Val Acc: 0.965598
30 Loss: 0.0721167 Train Acc: 0.9784 Val Acc: 0.97498
40 Loss: 0.0561971 Train Acc: 0.9864 Val Acc: 0.981626
50 Loss: 0.0423403 Train Acc: 0.9899 Val Acc: 0.984754
60 Loss: 0.0326954 Train Acc: 0.9929 Val Acc: 0.986317
70 Loss: 0.0254659 Train Acc: 0.9952 Val Acc: 0.986708
80 Loss: 0.0181217 Train Acc: 0.9971 Val Acc: 0.98749
90 Loss: 0.0146732 Train Acc: 0.9977 Val Acc: 0.989054
100 Loss: 0.010428 Train Acc: 0.9991 Val Acc: 0.989054
110 Loss: 0.00806561 Train Acc: 0.9995 Val Acc: 0.987881
120 Loss: 0.0067298 Train Acc: 0.9996 Val Acc: 0.988272
Eearly Stopping..


## 연습문제 9번 - transfer learning 

In [3]:
# 이번에는 데이터 5~9까지, 숫자별 100개씩
mask = mnist.train.labels > 4
X_train = np.empty([0,28*28])
y_train = np.empty(0)

for i in range(5,10):
    mask = mnist.train.labels == i
    X_temp = mnist.train.images[mask]
    y_temp = mnist.train.labels[mask]
    X_temp = X_temp[0:100]
    y_temp = y_temp[0:100]
    X_train = np.concatenate((X_train, X_temp))
    y_train = np.concatenate((y_train, y_temp))
    
y_train = y_train - 5

mask_val = mnist.validation.labels > 4
X_val = mnist.validation.images[mask_val]
y_val = mnist.validation.labels[mask_val]

y_val = y_val - 5

print(X_train.shape, y_train.shape)

# 그래프 환경설정
n_train_samples = X_train.shape[0]
input_dim  = X_train.shape[-1]
n_hidden = 100
n_class = 5

learning_rate = 0.001

(500, 784) (500,)


In [127]:
# 그래프 만들기

tf.reset_default_graph()

with tf.name_scope("input"):
    X = tf.placeholder(dtype=tf.float32, shape=(None, input_dim), name="X")
    y = tf.placeholder(dtype=tf.int64, shape=(None), name="y")
    is_training = tf.placeholder(tf.bool, shape=(), name="is_training")

from functools import partial 
he_init = tf.contrib.layers.variance_scaling_initializer()

layer_params = {
    "units": n_hidden,
    "activation": tf.nn.elu,
    "kernel_initializer": he_init
}
bn_params = {
    'training': is_training,
    'momentum': 0.99
}
def dense_with_bn_and_dropout_layer(inputs, bn_params, units, activation, kernel_initializer, name):
    with tf.variable_scope(name):
        dense = tf.layers.dense(inputs, units=units, activation=None, 
                        kernel_initializer=kernel_initializer, name="dense")
        norm = tf.layers.batch_normalization(dense, name="batchnorm", **bn_params)
        outputs = activation(norm, name="activation")
        return tf.layers.dropout(outputs) # 이렇게 넣어주면 되겠지?

custom_hidden_layer = partial(dense_with_bn_and_dropout_layer, bn_params=bn_params, **layer_params)

hidden1 = custom_hidden_layer(inputs=X, name="hidden1")
hidden2 = custom_hidden_layer(hidden1, name="hidden2")
hidden3 = custom_hidden_layer(hidden2, name="hidden3")
hidden4 = custom_hidden_layer(hidden3, name="hidden4")
hidden5 = custom_hidden_layer(hidden4, name="hidden5")
logits = tf.layers.dense(hidden5, n_class, name="logits_output")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, 
                                                              logits=logits, name="cross-entropy")
    loss = tf.reduce_mean(xentropy, name = "loss")

with tf.name_scope("eval"):
    y_pred = tf.argmax(logits, axis=1, name="prediction")
    correct = tf.equal(y_pred, y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
with tf.name_scope("training"):
    global_step = tf.train.get_or_create_global_step()
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        # logits_output 빼고 동결
        train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                       scope="logits_output")
        training_op = optimizer.minimize(loss, var_list=train_vars, 
                                         global_step=global_step)
        


In [128]:
## 학습
# 환경 설정
n_epoch = 1000
batch_size = 100

# Summary
from datetime import datetime
now = datetime.now().strftime("%Y%m%d%H%M%S")
logdir = "./ch11/log/run-{}".format(now)
loss_summary = tf.summary.scalar("loss", loss) # scalar 를 그래프에 먼저 추가해주고 file_writer 만들어야 하는듯..? 아닌가?
with tf.name_scope("accuracy"):
    val_acc_summary = tf.summary.scalar("validation", accuracy)
    train_acc_summary = tf.summary.scalar("train", accuracy)
file_writer = tf.summary.FileWriter(logdir = logdir, graph = tf.get_default_graph()) 

init = tf.global_variables_initializer()
reuse_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                              scope="hidden[12345]") # scope 가 이야기하는건 name_scope 맞는듯, regex
original_saver = tf.train.Saver(reuse_vars)
new_saver = tf.train.Saver()

In [129]:
with tf.Session() as sess:
    sess.run(init)
    original_saver.restore(sess,"ch11/model/practice8_best_model.ckpt")
    
    distance_to_best = 0
    best_val_acc = 0.0
    for epoch in range(n_epoch):
        new_idx = np.random.permutation(n_train_samples)
        X_train = X_train[new_idx]
        y_train = y_train[new_idx]
        
        for batch in range( n_train_samples // batch_size ):
            start = batch * batch_size
            end = (batch+1) * batch_size
            feed_dict = {X: X_train[start:end], y: y_train[start:end], is_training: True}
            sess.run(training_op, feed_dict=feed_dict)
            
            # tensorboard summary 기록
            if batch % 10 == 0:
                loss_val, train_acc_summ_eval = sess.run([loss_summary, train_acc_summary], feed_dict=feed_dict)
                file_writer.add_summary(loss_val, tf.train.global_step(sess, global_step))
                file_writer.add_summary(train_acc_summ_eval, tf.train.global_step(sess, global_step))
        
        # Accuracy Evaluation
        if epoch % 10 == 0:
            feed_dict[is_training] = False
            loss_eval, train_acc_eval, val_acc_summ_eval = sess.run([loss, accuracy, val_acc_summary], feed_dict=feed_dict)
            val_acc_eval = accuracy.eval({X:X_val, y:y_val, is_training:False})
            file_writer.add_summary(val_acc_summ_eval, tf.train.global_step(sess, global_step))
            
            print(epoch, "Loss:", loss_eval, "Train Acc:", train_acc_eval, "Val Acc:", val_acc_eval)
            if val_acc_eval > best_val_acc :
                distance_to_best = 0
                best_val_acc = val_acc_eval
                save_path = new_saver.save(sess, "ch11/model/practice8_transfer_best_model.ckpt")
            else:
                distance_to_best += 1
        # 조기 종료
        if distance_to_best > 2:
            print("Eearly Stopping..")
            break

    save_path = saver.save(sess, "ch11/model/practice8_transfer_model_final.ckpt")
    file_writer.close() # 꼭 닫아줘야 텐서보드에 잘 나타나는듯

INFO:tensorflow:Restoring parameters from ch11/model/practice8_best_model.ckpt
0 Loss: 2.09043 Train Acc: 0.17 Val Acc: 0.158067
10 Loss: 1.09095 Train Acc: 0.57 Val Acc: 0.577805
20 Loss: 0.687687 Train Acc: 0.83 Val Acc: 0.708436
30 Loss: 0.584457 Train Acc: 0.82 Val Acc: 0.75389
40 Loss: 0.546494 Train Acc: 0.84 Val Acc: 0.785012
50 Loss: 0.491019 Train Acc: 0.83 Val Acc: 0.807535
60 Loss: 0.560905 Train Acc: 0.85 Val Acc: 0.817363
70 Loss: 0.525453 Train Acc: 0.87 Val Acc: 0.822277
80 Loss: 0.45224 Train Acc: 0.87 Val Acc: 0.82801
90 Loss: 0.366214 Train Acc: 0.91 Val Acc: 0.829648
100 Loss: 0.373929 Train Acc: 0.9 Val Acc: 0.834152
110 Loss: 0.332612 Train Acc: 0.91 Val Acc: 0.8362
120 Loss: 0.344139 Train Acc: 0.88 Val Acc: 0.837428
130 Loss: 0.261225 Train Acc: 0.95 Val Acc: 0.840295
140 Loss: 0.265381 Train Acc: 0.96 Val Acc: 0.841114
150 Loss: 0.279217 Train Acc: 0.92 Val Acc: 0.840704
160 Loss: 0.275937 Train Acc: 0.95 Val Acc: 0.84398
170 Loss: 0.217167 Train Acc: 0.95 Val A

#### Caching

중간에 stop gradient 층 넣어줘야함 => 그래도 안되네.. 왜안될까..

In [134]:
# 그래프 만들기

tf.reset_default_graph()

with tf.name_scope("input"):
    X = tf.placeholder(dtype=tf.float32, shape=(None, input_dim), name="X")
    y = tf.placeholder(dtype=tf.int64, shape=(None), name="y")
    is_training = tf.placeholder(tf.bool, shape=(), name="is_training")

from functools import partial 
he_init = tf.contrib.layers.variance_scaling_initializer()

layer_params = {
    "units": n_hidden,
    "activation": tf.nn.elu,
    "kernel_initializer": he_init
}
bn_params = {
    'training': is_training,
    'momentum': 0.99
}
def dense_with_bn_and_dropout_layer(inputs, bn_params, units, activation, kernel_initializer, name):
    with tf.variable_scope(name):
        dense = tf.layers.dense(inputs, units=units, activation=None, 
                        kernel_initializer=kernel_initializer, name="dense")
        norm = tf.layers.batch_normalization(dense, name="batchnorm", **bn_params)
        outputs = activation(norm, name="activation")
        return tf.layers.dropout(outputs) # 이렇게 넣어주면 되겠지?

custom_hidden_layer = partial(dense_with_bn_and_dropout_layer, bn_params=bn_params, **layer_params)

hidden1 = custom_hidden_layer(inputs=X, name="hidden1")
hidden2 = custom_hidden_layer(hidden1, name="hidden2")
hidden3 = custom_hidden_layer(hidden2, name="hidden3")
hidden4 = custom_hidden_layer(hidden3, name="hidden4")
hidden5 = custom_hidden_layer(hidden4, name="hidden5")
hidden5_stop = tf.stop_gradient(hidden5)
logits = tf.layers.dense(hidden5_stop, n_class, name="logits_output")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, 
                                                              logits=logits, name="cross-entropy")
    loss = tf.reduce_mean(xentropy, name = "loss")

with tf.name_scope("eval"):
    y_pred = tf.argmax(logits, axis=1, name="prediction")
    correct = tf.equal(y_pred, y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
with tf.name_scope("training"):
    global_step = tf.train.get_or_create_global_step()
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        # logits_output 빼고 동결
        train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                       scope="logits_output")
        training_op = optimizer.minimize(loss, var_list=train_vars, 
                                         global_step=global_step)
        


In [135]:
## 학습
# 환경 설정
n_epoch = 1000
batch_size = 100

# Summary
from datetime import datetime
now = datetime.now().strftime("%Y%m%d%H%M%S")
logdir = "./ch11/log/run-{}".format(now)
loss_summary = tf.summary.scalar("loss", loss) # scalar 를 그래프에 먼저 추가해주고 file_writer 만들어야 하는듯..? 아닌가?
with tf.name_scope("accuracy"):
    val_acc_summary = tf.summary.scalar("validation", accuracy)
    train_acc_summary = tf.summary.scalar("train", accuracy)
file_writer = tf.summary.FileWriter(logdir = logdir, graph = tf.get_default_graph()) 

init = tf.global_variables_initializer()
reuse_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                              scope="hidden[12345]") # scope 가 이야기하는건 name_scope 맞는듯, regex
original_saver = tf.train.Saver(reuse_vars)
new_saver = tf.train.Saver()

In [136]:


with tf.Session() as sess:
    sess.run(init)
    original_saver.restore(sess,"ch11/model/practice8_best_model.ckpt")
    
    distance_to_best = 0
    best_val_acc = 0.0
    
    # caching
    hidden5_cache = sess.run(hidden5, feed_dict={X: X_train, is_training: True})
    
    for epoch in range(n_epoch):
        new_idx = np.random.permutation(n_train_samples)
        hidden5_cache = hidden5_cache[new_idx]
        y_train = y_train[new_idx]
        
        for batch in range( n_train_samples // batch_size ):
            start = batch * batch_size
            end = (batch+1) * batch_size
            feed_dict = {hidden5: hidden5_cache[start:end], y: y_train[start:end], is_training: True}
            sess.run(training_op, feed_dict=feed_dict)
            
            # tensorboard summary 기록
            if batch % 10 == 0:
                loss_val, train_acc_summ_eval = sess.run([loss_summary, train_acc_summary], feed_dict=feed_dict)
                file_writer.add_summary(loss_val, tf.train.global_step(sess, global_step))
                file_writer.add_summary(train_acc_summ_eval, tf.train.global_step(sess, global_step))
        
        # Accuracy Evaluation
        if epoch % 10 == 0:
            feed_dict[is_training] = False
            loss_eval, train_acc_eval, val_acc_summ_eval = sess.run([loss, accuracy, val_acc_summary], feed_dict=feed_dict)
            val_acc_eval = accuracy.eval({X:X_val, y:y_val, is_training:False})
            file_writer.add_summary(val_acc_summ_eval, tf.train.global_step(sess, global_step))
            
            print(epoch, "Loss:", loss_eval, "Train Acc:", train_acc_eval, "Val Acc:", val_acc_eval)
            if val_acc_eval > best_val_acc :
                distance_to_best = 0
                best_val_acc = val_acc_eval
                save_path = new_saver.save(sess, "ch11/model/practice8_transfer_best_model.ckpt")
            else:
                distance_to_best += 1
        # 조기 종료
        if distance_to_best > 2:
            print("Eearly Stopping..")
            break

    save_path = saver.save(sess, "ch11/model/practice8_transfer_model_final.ckpt")
    file_writer.close() # 꼭 닫아줘야 텐서보드에 잘 나타나는듯

INFO:tensorflow:Restoring parameters from ch11/model/practice8_best_model.ckpt


InvalidArgumentError: You must feed a value for placeholder tensor 'input/X' with dtype float and shape [?,784]
	 [[Node: input/X = Placeholder[dtype=DT_FLOAT, shape=[?,784], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'input/X', defined at:
  File "/Users/JOLEE/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/JOLEE/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/JOLEE/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/JOLEE/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/JOLEE/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/JOLEE/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, 

#### 1~4층만 동결

성능이 좀 더 나은듯

In [130]:
# 그래프 만들기

tf.reset_default_graph()

with tf.name_scope("input"):
    X = tf.placeholder(dtype=tf.float32, shape=(None, input_dim), name="X")
    y = tf.placeholder(dtype=tf.int64, shape=(None), name="y")
    is_training = tf.placeholder(tf.bool, shape=(), name="is_training")

from functools import partial 
he_init = tf.contrib.layers.variance_scaling_initializer()

layer_params = {
    "units": n_hidden,
    "activation": tf.nn.elu,
    "kernel_initializer": he_init
}
bn_params = {
    'training': is_training,
    'momentum': 0.99
}
def dense_with_bn_and_dropout_layer(inputs, bn_params, units, activation, kernel_initializer, name):
    with tf.variable_scope(name):
        dense = tf.layers.dense(inputs, units=units, activation=None, 
                        kernel_initializer=kernel_initializer, name="dense")
        norm = tf.layers.batch_normalization(dense, name="batchnorm", **bn_params)
        outputs = activation(norm, name="activation")
        return tf.layers.dropout(outputs) # 이렇게 넣어주면 되겠지?

custom_hidden_layer = partial(dense_with_bn_and_dropout_layer, bn_params=bn_params, **layer_params)

hidden1 = custom_hidden_layer(inputs=X, name="hidden1")
hidden2 = custom_hidden_layer(hidden1, name="hidden2")
hidden3 = custom_hidden_layer(hidden2, name="hidden3")
hidden4 = custom_hidden_layer(hidden3, name="hidden4")
hidden5 = custom_hidden_layer(hidden4, name="hidden5")
logits = tf.layers.dense(hidden5, n_class, name="logits_output")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, 
                                                              logits=logits, name="cross-entropy")
    loss = tf.reduce_mean(xentropy, name = "loss")

with tf.name_scope("eval"):
    y_pred = tf.argmax(logits, axis=1, name="prediction")
    correct = tf.equal(y_pred, y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
with tf.name_scope("training"):
    global_step = tf.train.get_or_create_global_step()
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        # logits_output 빼고 동결
        train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                       scope="hidden5|logits_output")
        training_op = optimizer.minimize(loss, var_list=train_vars, 
                                         global_step=global_step)
        


In [131]:
## 학습
# 환경 설정
n_epoch = 1000
batch_size = 100

# Summary
from datetime import datetime
now = datetime.now().strftime("%Y%m%d%H%M%S")
logdir = "./ch11/log/run-{}".format(now)
loss_summary = tf.summary.scalar("loss", loss) # scalar 를 그래프에 먼저 추가해주고 file_writer 만들어야 하는듯..? 아닌가?
with tf.name_scope("accuracy"):
    val_acc_summary = tf.summary.scalar("validation", accuracy)
    train_acc_summary = tf.summary.scalar("train", accuracy)
file_writer = tf.summary.FileWriter(logdir = logdir, graph = tf.get_default_graph()) 

init = tf.global_variables_initializer()
reuse_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                              scope="hidden[12345]") # scope 가 이야기하는건 name_scope 맞는듯, regex
original_saver = tf.train.Saver(reuse_vars)
new_saver = tf.train.Saver()

In [133]:
with tf.Session() as sess:
    sess.run(init)
    original_saver.restore(sess,"ch11/model/practice8_best_model.ckpt")
    
    distance_to_best = 0
    best_val_acc = 0.0
    for epoch in range(n_epoch):
        new_idx = np.random.permutation(n_train_samples)
        X_train = X_train[new_idx]
        y_train = y_train[new_idx]
        
        for batch in range( n_train_samples // batch_size ):
            start = batch * batch_size
            end = (batch+1) * batch_size
            feed_dict = {X: X_train[start:end], y: y_train[start:end], is_training: True}
            sess.run(training_op, feed_dict=feed_dict)
            
            # tensorboard summary 기록
            if batch % 10 == 0:
                loss_val, train_acc_summ_eval = sess.run([loss_summary, train_acc_summary], feed_dict=feed_dict)
                file_writer.add_summary(loss_val, tf.train.global_step(sess, global_step))
                file_writer.add_summary(train_acc_summ_eval, tf.train.global_step(sess, global_step))
        
        # Accuracy Evaluation
        if epoch % 10 == 0:
            feed_dict[is_training] = False
            loss_eval, train_acc_eval, val_acc_summ_eval = sess.run([loss, accuracy, val_acc_summary], feed_dict=feed_dict)
            val_acc_eval = accuracy.eval({X:X_val, y:y_val, is_training:False})
            file_writer.add_summary(val_acc_summ_eval, tf.train.global_step(sess, global_step))
            
            print(epoch, "Loss:", loss_eval, "Train Acc:", train_acc_eval, "Val Acc:", val_acc_eval)
            if val_acc_eval > best_val_acc :
                distance_to_best = 0
                best_val_acc = val_acc_eval
                save_path = new_saver.save(sess, "ch11/model/practice8_transfer_best_model.ckpt")
            else:
                distance_to_best += 1
        # 조기 종료
        if distance_to_best > 2:
            print("Eearly Stopping..")
            break

    save_path = saver.save(sess, "ch11/model/practice8_transfer_model_final.ckpt")
    file_writer.close() # 꼭 닫아줘야 텐서보드에 잘 나타나는듯

INFO:tensorflow:Restoring parameters from ch11/model/practice8_best_model.ckpt
0 Loss: 1.70593 Train Acc: 0.29 Val Acc: 0.269861
10 Loss: 0.449963 Train Acc: 0.87 Val Acc: 0.814906
20 Loss: 0.303471 Train Acc: 0.9 Val Acc: 0.845209
30 Loss: 0.252559 Train Acc: 0.9 Val Acc: 0.861179
40 Loss: 0.103997 Train Acc: 0.99 Val Acc: 0.875102
50 Loss: 0.135623 Train Acc: 0.97 Val Acc: 0.880426
60 Loss: 0.0874422 Train Acc: 0.99 Val Acc: 0.883292
70 Loss: 0.0534014 Train Acc: 1.0 Val Acc: 0.882883
80 Loss: 0.0456853 Train Acc: 1.0 Val Acc: 0.885749
90 Loss: 0.0376235 Train Acc: 1.0 Val Acc: 0.883292
100 Loss: 0.0268364 Train Acc: 1.0 Val Acc: 0.883292
110 Loss: 0.0217215 Train Acc: 1.0 Val Acc: 0.886978
120 Loss: 0.015971 Train Acc: 1.0 Val Acc: 0.883292
130 Loss: 0.0132684 Train Acc: 1.0 Val Acc: 0.88493
140 Loss: 0.0121794 Train Acc: 1.0 Val Acc: 0.884111
Eearly Stopping..


#### 1~3층만 동결


In [92]:
# 그래프 만들기

tf.reset_default_graph()

with tf.name_scope("input"):
    X = tf.placeholder(dtype=tf.float32, shape=(None, input_dim), name="X")
    y = tf.placeholder(dtype=tf.int64, shape=(None), name="y")
    is_training = tf.placeholder(tf.bool, shape=(), name="is_training")

from functools import partial 
he_init = tf.contrib.layers.variance_scaling_initializer()

layer_params = {
    "units": n_hidden,
    "activation": tf.nn.elu,
    "kernel_initializer": he_init
}
bn_params = {
    'training': is_training,
    'momentum': 0.99
}
def dense_with_bn_and_dropout_layer(inputs, bn_params, units, activation, kernel_initializer, name):
    with tf.variable_scope(name):
        dense = tf.layers.dense(inputs, units=units, activation=None, 
                        kernel_initializer=kernel_initializer, name="dense")
        norm = tf.layers.batch_normalization(dense, name="batchnorm", **bn_params)
        outputs = activation(norm, name="activation")
        return tf.layers.dropout(outputs) # 이렇게 넣어주면 되겠지?

custom_hidden_layer = partial(dense_with_bn_and_dropout_layer, bn_params=bn_params, **layer_params)

hidden1 = custom_hidden_layer(inputs=X, name="hidden1")
hidden2 = custom_hidden_layer(hidden1, name="hidden2")
hidden3 = custom_hidden_layer(hidden2, name="hidden3")
hidden4 = custom_hidden_layer(hidden3, name="hidden4")
hidden5 = custom_hidden_layer(hidden4, name="hidden5")
logits = tf.layers.dense(hidden5, n_class, name="logits_output")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, 
                                                              logits=logits, name="cross-entropy")
    loss = tf.reduce_mean(xentropy, name = "loss")

with tf.name_scope("eval"):
    y_pred = tf.argmax(logits, axis=1, name="prediction")
    correct = tf.equal(y_pred, y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
with tf.name_scope("training"):
    global_step = tf.train.get_or_create_global_step()
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        # logits_output 빼고 동결
        train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                       scope="hidden[45]|logits_output")
        training_op = optimizer.minimize(loss, var_list=train_vars, 
                                         global_step=global_step)
        


In [93]:
## 학습
# 환경 설정
n_epoch = 1000
batch_size = 100

# Summary
from datetime import datetime
now = datetime.now().strftime("%Y%m%d%H%M%S")
logdir = "./ch11/log/run-{}".format(now)
loss_summary = tf.summary.scalar("loss", loss) # scalar 를 그래프에 먼저 추가해주고 file_writer 만들어야 하는듯..? 아닌가?
with tf.name_scope("accuracy"):
    val_acc_summary = tf.summary.scalar("validation", accuracy)
    train_acc_summary = tf.summary.scalar("train", accuracy)
file_writer = tf.summary.FileWriter(logdir = logdir, graph = tf.get_default_graph()) 

init = tf.global_variables_initializer()
reuse_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                              scope="hidden[12345]") # scope 가 이야기하는건 name_scope 맞는듯, regex
original_saver = tf.train.Saver(reuse_vars)
new_saver = tf.train.Saver()

In [94]:
with tf.Session() as sess:
    sess.run(init)
    original_saver.restore(sess,"ch11/model/practice8_best_model.ckpt")
    
    distance_to_best = 0
    best_val_acc = 0.0
    for epoch in range(n_epoch):
        new_idx = np.random.permutation(n_train_samples)
        X_train = X_train[new_idx]
        y_train = y_train[new_idx]
        
        for batch in range( n_train_samples // batch_size ):
            start = batch * batch_size
            end = (batch+1) * batch_size
            feed_dict = {X: X_train[start:end], y: y_train[start:end], is_training: True}
            sess.run(training_op, feed_dict=feed_dict)
            
            # tensorboard summary 기록
            if batch % 10 == 0:
                loss_val, train_acc_summ_eval = sess.run([loss_summary, train_acc_summary], feed_dict=feed_dict)
                file_writer.add_summary(loss_val, tf.train.global_step(sess, global_step))
                file_writer.add_summary(train_acc_summ_eval, tf.train.global_step(sess, global_step))
        
        # Accuracy Evaluation
        if epoch % 10 == 0:
            feed_dict[is_training] = False
            loss_eval, train_acc_eval, val_acc_summ_eval = sess.run([loss, accuracy, val_acc_summary], feed_dict=feed_dict)
            val_acc_eval = accuracy.eval({X:X_val, y:y_val, is_training:False})
            file_writer.add_summary(val_acc_summ_eval, tf.train.global_step(sess, global_step))
            
            print(epoch, "Loss:", loss_eval, "Train Acc:", train_acc_eval, "Val Acc:", val_acc_eval)
            if val_acc_eval > best_val_acc :
                distance_to_best = 0
                best_val_acc = val_acc_eval
                save_path = new_saver.save(sess, "ch11/model/practice8_transfer_best_model.ckpt")
            else:
                distance_to_best += 1
        # 조기 종료
        if distance_to_best > 2:
            print("Eearly Stopping..")
            break

    save_path = saver.save(sess, "ch11/model/practice8_transfer_model_final.ckpt")
    file_writer.close() # 꼭 닫아줘야 텐서보드에 잘 나타나는듯

INFO:tensorflow:Restoring parameters from ch11/model/practice8_best_model.ckpt
0 Loss: 1.57544 Train Acc: 0.34 Val Acc: 0.305897
10 Loss: 0.476768 Train Acc: 0.87 Val Acc: 0.790745
20 Loss: 0.284394 Train Acc: 0.91 Val Acc: 0.830467
30 Loss: 0.233387 Train Acc: 0.92 Val Acc: 0.850532
40 Loss: 0.181986 Train Acc: 0.96 Val Acc: 0.864865
50 Loss: 0.113847 Train Acc: 1.0 Val Acc: 0.873874
60 Loss: 0.0758464 Train Acc: 1.0 Val Acc: 0.876331
70 Loss: 0.0549482 Train Acc: 1.0 Val Acc: 0.879607
80 Loss: 0.0453231 Train Acc: 1.0 Val Acc: 0.880016
90 Loss: 0.0385064 Train Acc: 1.0 Val Acc: 0.880835
100 Loss: 0.0400328 Train Acc: 1.0 Val Acc: 0.87674
110 Loss: 0.0198033 Train Acc: 1.0 Val Acc: 0.881654
120 Loss: 0.0229281 Train Acc: 1.0 Val Acc: 0.879197
130 Loss: 0.0150742 Train Acc: 1.0 Val Acc: 0.882883
140 Loss: 0.0145076 Train Acc: 1.0 Val Acc: 0.880426
150 Loss: 0.0116535 Train Acc: 1.0 Val Acc: 0.883702
160 Loss: 0.0130467 Train Acc: 1.0 Val Acc: 0.880426
170 Loss: 0.0117968 Train Acc: 1.0